In [1]:
(asdf:load-system :cram-pr2-pick-place-demo)
(roslisp-utilities:startup-ros)
(defpackage cram-bullet-world-tutorial
  (:nicknames :btw-tut)
  (:use :common-lisp :cram-prolog :desig :exe))
(in-package :btw-tut)

(defun make-pose (frame point euler)
    (declare (type string frame)
             (type list point euler))
    "Creates a cl-transforms pose in 3D space w.r.t. the given frame, point and euler."
    (if (and (eq (length point) 3) (eq (length euler) 3))
        (destructuring-bind (ax ay az) (mapcar #'eval euler)
          (cl-transforms-stamped:make-pose-stamped frame 0.0 
                                                   (apply #'cl-transforms:make-3d-vector point)
                                                   (cl-transforms:euler->quaternion :ax ax :ay ay :az az)))
        (format T "point ~a or euler ~a is not of length 3" point euler)))

(defun simulate (&key (duration 5.0)    ;; in seconds 
                      (framerate 20.0)) ;; in Hz
    (let ((sleep-interval (/ 1.0 framerate))
          (times (floor (* duration framerate))))
         (dotimes (_ times)
             (sleep (/ sleep-interval 2))
             (btr:simulate btr:*current-bullet-world* (/ sleep-interval 2)))))

(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

T

NIL

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

MAKE-POSE

...

*BASE-POSE-NEAR-TABLE*

*DOWNWARD-LOOK-COORDINATE*

*OBJECT-DELIVERY-POSE*

*BASE-POSE-NEAR-KITCHEN-ISLAND*

BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/cl_store/src/cl-store.asd" contains definition for system "cl-store-tests". Please only define "cl-store" and secondary systems with a name starting with "cl-store/" (e.g. "cl-store/test") in that file.
BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/split_sequence/src/split-sequence.asd" contains definition for system "split-sequence-tests". Please only define "split-sequence" and secondary systems with a name starting with "split-sequence/" (e.g. "split-sequence/test") in that file.
BAD-SYSTEM-NAME: System definition file #P"/home/workspace/ros/src/cram/cram_3rdparty/cl_ppcre/src/cl-ppcre.asd" contains definition for system "cl-ppcre-test". Please only define "cl-ppcre" and secondary systems with a name starting with "cl-ppcre/" (e.g. "cl-ppcre/test") in that file.
SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CRAM-MANIPULATION-INTERFACES::SYMBOL-TO-PROLOG-RULE 

In [2]:
(defun pick (?object ?nav-goal ?poi &key ?arm)
    (let ((?arm (if (or (listp ?arm) (not ?arm))
                    ?arm
                    `(,?arm)))
          ?perceived-object)
       (cpl:par
             ;; Park arms
             (perform (an action
                          (type parking-arms)))
             ;; Move the torso up
             (perform (a motion 
                         (type moving-torso)
                         (joint-angle 0.3)))
             ;; Go to the navigation goal
             (perform (an action
                          (type going)
                          (target (a location 
                                     (pose ?nav-goal))))))
         ;; Look at the point of interest
         (perform (an action
                      (type looking)
                      (target (a location 
                                 (pose ?poi)))))
         ;; Detect the object
         (setf ?perceived-object 
               (perform (an action
                            (type detecting)
                            (object ?object))))
         ;; Pick up the detected object
         (perform (an action
                      (type picking-up)
                      (desig:when ?arm
                          (arm ?arm))
                      (object ?perceived-object)))
         ;; Park arms
         (perform (an action
                      (type parking-arms)))
         ?perceived-object))

PICK

In [3]:
(defun place (?nav-goal ?drop-pose &key ?object ?arm)
    (let ((?arm (if (or (listp ?arm) 
                        (not ?arm))
                    ?arm
                    `(,?arm))))
         ;; Move the torso up so the robot can reach the higher surface better
         ;; without crashing his arms into it.
         (perform (a motion 
                     (type moving-torso)
                     (joint-angle 0.3)))
         ;; Go to the target delivery location
         (perform (an action
                      (type going)
                      (target (a location
                                 (pose ?nav-goal)))))
         ;; Place down the bottle
         (perform (an action
                      (type placing)
                      (desig:when ?arm
                          (arm ?arm))
                      (desig:when ?object
                          (object ?object))
                      (target (a location
                                 (pose ?drop-pose)))))
         ;; Park arms
         (perform (an action
                      (type parking-arms)))))

PLACE

In [12]:
;(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))

(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 1.2) '((* 1.5 pi) (* 0.5 pi) pi)))


*OBJECT-DELIVERY-POSE*

In [13]:
;; Reset the world
(demo::initialize)

(loop while t
    do 
      (btr-utils:kill-all-objects)
    ;; Spawn the bottle
    (btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)
    
    (urdf-proj:with-simulated-robot
     (let ((?drop-pose *object-delivery-pose*)
           ?perceived-object)

          (pick (an object (type bottle))
                *base-pose-near-table*
                *downward-look-coordinate*
                :?arm :right)

          (place *base-pose-near-kitchen-island*
                 *object-delivery-pose*))) 

        (simulate))

NIL

interrupt: Execution interrupted

[(SET-ROBOT-STATE-FROM-TF) WARN] 1664891738.674: Failed with transform-stamped-error:
    No transform was published between frames base_footprint and map
    Ignore this warning if no real robot is running.
SB-INT:SIMPLE-STYLE-WARNING: The variable ?DROP-POSE is defined but never used.
SB-INT:SIMPLE-STYLE-WARNING: The variable ?PERCEIVED-OBJECT is defined but never used.


In [ ]:
;; Reset the world
(demo::initialize)

;; Have all parameters we need in one place.
(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

;; Spawn the bottle
(btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)

(urdf-proj:with-simulated-robot

 (let ((?nav-goal *base-pose-near-kitchen-island*)
       (?looking-pose *downward-look-coordinate*)
       (?drop-pose *object-delivery-pose*)
       ?perceived-bottle)
      
      (setf ?perceived-bottle
            (pick (an object (type bottle))
                  *base-pose-near-table*
                  *downward-look-coordinate*))
               
      
      ;; Move the torso up so the robot can reach the higher surface better
      ;; without crashing his arms into it.
      (perform (a motion 
                  (type moving-torso)
                  (joint-angle 0.3)))
      ;; Go to the target delivery location
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?nav-goal)))))
      ;; Place down the bottle
      (perform (an action
                   (type placing)
                   (arm (right))
                   ;; here we explicitly specify to put down the bottle
                   ;; it's not necessary, but you can use this as a reference for the upcoming task
                   (object ?perceived-bottle) 
                   (target (a location
                              (pose ?drop-pose)))))
            ;; Park arms
      (perform (an action
                   (type parking-arms)))))

In [ ]:
;; Reset the world
(demo::initialize)

;; Have all parameters we need in one place.
(defparameter *bottle-spawn-pose* (make-pose "map" '(-2.323 -1 0.82) '(0 0 0)))
(defparameter *base-pose-near-table* (make-pose "map" '(-2.2 -0.2 0.0) '(0.0 0.0 (* pi -0.5))))
(defparameter *downward-look-coordinate* (make-pose "base_footprint" '(0.7 0.0 1.0) '(0 0 0)))
(defparameter *object-delivery-pose* (make-pose "map" '(-0.8 2 0.9) '(0 0 0)))
(defparameter *base-pose-near-kitchen-island* (make-pose "map" '(-0.15 2.0 0.0) '(0.0 0.0 pi)))

;; Spawn the bottle
(btr-utils:spawn-object 'bottle-1 :bottle :pose *bottle-spawn-pose*)

(urdf-proj:with-simulated-robot
 ;; Go to the table where the bottle stands
 (let ((?navigation-goal *base-pose-near-table*))
      (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal))))))
 ;; Look down at the bottle
 (let ((?looking-direction *downward-look-coordinate*))
      (perform (an action
                   (type looking)
                   (target (a location 
                              (pose ?looking-direction))))))
 ;; Detect the bottle and keep the object designator in ?perceived-bottle
 ;; so we can use it later
 (let ((?nav-goal *base-pose-near-kitchen-island*)
       (?drop-pose *object-delivery-pose*)
       (?perceived-bottle (perform (an action
                                       (type detecting)
                                       (object (an object
                                                   (type bottle)))))))
      ;; Pick up the detected bottle
      (perform (an action
                   (type picking-up)
                   (arm (right))
                   (object ?perceived-bottle)))
      ;; Park arms
      (perform (an action
                   (type parking-arms)))
      ;; Move the torso up so the robot can reach the higher surface better
      ;; without crashing his arms into it.
      (perform (a motion 
                  (type moving-torso)
                  (joint-angle 0.3)))
      ;; Go to the target delivery location
      (perform (an action
                   (type going)
                   (target (a location
                              (pose ?nav-goal)))))
      ;; Place down the bottle
      (perform (an action
                   (type placing)
                   (arm (right))
                   ;; here we explicitly specify to put down the bottle
                   ;; it's not necessary, but you can use this as a reference for the upcoming task
                   (object ?perceived-bottle) 
                   (target (a location
                              (pose ?drop-pose)))))
            ;; Park arms
      (perform (an action
                   (type parking-arms)))))